# Other Prompts

### Prompt for Generating LLM-based Examples (Few-Shot Prompting)

To generate LLM-based examples, we utilized a locally-hosted `LLaMa-2-7B-chat-hf` model, which was prompted with all 80 queries. For experimental consistency and to ensure direct comparability with the human-generated examples, we only included LLM-generated responses that corresponded to the same queries used in the human approach. The complete set of responses used in our experiments is available in the `few_shot.ipynb` notebook for reference and reproducibility.

### Load Queries

In [ ]:
import util

all_factual_queries = util.load_file('Queries/factual-queries.txt')
factual_queries = all_factual_queries[10:50]

all_subjective_queries = util.load_file('Queries/subjective-queries.txt')
subjective_queries = all_subjective_queries[10:50]

print(F"{len(factual_queries)} factual queries have been loaded.")
print(F"{len(subjective_queries)} subjective queries have been loaded.")

In [ ]:
import transformers
import torch
import re

def chat(cl, ml=1000):
    inputs = llama_tokenizer(cl, return_tensors="pt")
    generate_ids = llama_model.generate(inputs.input_ids, max_length=ml).to(DEVICE)

    cl_new = llama_tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 

    return(cl_new)

model_path = "ADD MODEL PATH"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

llama_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
llama_model = transformers.AutoModelForCausalLM.from_pretrained(model_path).to(DEVICE)
llama_model.to(DEVICE)

emotions = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
persona = "You are a helpful and honest assistant, skilled in writing emotional texts."


########################################## FACTUAL

responses = []
emotion_list = []
queries = []

for query in factual_queries:
    for emotion in emotions:
        print("Started generating responses.")

        chat_log = F"""
            [SYS]{persona}[/SYS] 
            [INST]
                ###Instruction###: Answer the following question in the style of the given emotion.

                ###Question###: {query} 

                ###Emotion###: {emotion}
            [\INST]
            """
                
        new_chat_log = chat(chat_log)
        response = new_chat_log.replace(chat_log, "")
        cleaned_response = re.sub(r'\n', '', response)

        responses.append(cleaned_response)
        emotion_list.append(emotion)
        queries.append(query)

output_path = "responses-llama_factual"
util.save_dataframe_files (responses, emotion_list, queries, output_path)

########################################## SUBJECTIVE

responses = []
emotion_list = []
queries = []

for query in subjective_queries:
    for emotion in emotions:
        print("Started generating responses.")

        chat_log = F"""
            [SYS]{persona}[/SYS] 
            [INST]
                ###Instruction###: Write a text of 100 words based on the following input in the style of the given emotion.

                ###Question###: {query} 

                ###Emotion###: {emotion}
            [\INST]
            """
                
        new_chat_log = chat(chat_log)
        response = new_chat_log.replace(chat_log, "")
        cleaned_response = re.sub(r'\n', '', response)

        responses.append(cleaned_response)
        emotion_list.append(emotion)
        queries.append(query)

output_path = "responses-llama_subjective"
util.save_dataframe_files (responses, emotion_list, queries, output_path)

### Prompt for Generating Reasoning Texts (CoT Prompting)

We provide the original prompt used for generating the reasoning texts with GPT-3.5-turbo. However, we also added the reasoning texts used in our experiments in the folder cot_examples (auto_factual_reasoning and auto_subjective_reasoning).

In [ ]:
import pandas as pd

df_factual_examples = pd.read_pickle("cot_examples/factual_examples")
df_subjective_examples = pd.read_pickle("cot_examples/subjective_examples")

output_path = "cot_exmamples/reasoning_texts"

df_examples = pd.concat([df_factual_examples, df_subjective_examples], axis=0)

responses = []

for _, row in df_examples:
  prompt = F"Explain how the emotion {row['label']} is expressed in the following text. {row['text']}"

  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user", "content": prompt}
    ],
    seed = 16,
    temperature = 0.0
  )

  responses.append(completion.choices[0].message.content)

pd.to_pickle(output_path)